#  CAFormer-S18
* CAFormer-S18 적용
    - caformer_s18.sail_in22k_ft_in1k_384
    - dropout, normalization 적용
* Augmentation
    - Transpose # 행렬 스왑
    - HorizontalFlip # 좌우 반전
    - VerticalFlip # 상하 반전
    - CoarseDrop
* lr scheduler
    - Custom CosineAnnealingWarmUpRestarts
    - T_hold : min_lr * 10 적용
* optimizer
    - AdamW
* focal loss 적용
* cutmix 
    - lam `0~0.25`, `0.75~1`
* kfold
* result
    - fold0 : Epoch [0-36], lr : [0.000001], Train Loss : [0.2117], Val Loss : [0.4140], Val F1 Score : [0.8261]
    - fold1 : Epoch [1-44], lr : [0.000004], Train Loss : [0.1957], Val Loss : [0.3691], Val F1 Score : [0.8332]
    - fold2 : Epoch [2-33], lr : [0.000003], Train Loss : [0.2024], Val Loss : [0.3810], Val F1 Score : [0.8270]
    - fold3 : Epoch [3-39], lr : [0.000003], Train Loss : [0.2342], Val Loss : [0.3681], Val F1 Score : [0.8416]
    - fold4 : Epoch [4-22], lr : [0.000004], Train Loss : [0.2593], Val Loss : [0.4221], Val F1 Score : [0.8260]
    - 41539
    - public 점수 : 0.8404591477
    - private 점수 : 0.8416979765

In [1]:
import gc
import os
import random
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import timm

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
CFG = {
    'IMG_SIZE': 272,
    'EPOCHS': 1000,
    'LEARNING_RATE': 5e-7,
    'BATCH_SIZE': 16,
    'PATIENCE': 12,
    'WARMUP': 6,
    'K-FOLD': 5,
    'FILENAME': 'kfold_caformer',
    'SEED': 6
}

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(CFG['SEED']) # Seed 고정

In [5]:
running_colab = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False
if running_colab:
    from google.colab import drive
    drive.mount('/content/drive')
if running_colab:
    data_path = '/content/drive/MyDrive/Colab Notebooks/ai6th/data/optiver/'
else:
    data_path = '../../data/'

In [6]:
df = pd.read_csv(os.path.join(data_path, 'train.csv'))
df.loc[3896, 'artist'] = 'Titian'
df.loc[3986, 'artist'] = 'Alfred Sisley'
df.head()

,id,img_path,artist
0,0,./train/0000.jpg,Diego Velazquez
1,1,./train/0001.jpg,Vincent van Gogh
2,2,./train/0002.jpg,Claude Monet
3,3,./train/0003.jpg,Edgar Degas
4,4,./train/0004.jpg,Hieronymus Bosch


In [7]:
artists = df.groupby('artist')[['id']].count().rename(columns={'id':'count'}).reset_index()

In [8]:
# Label Encoding
le = preprocessing.LabelEncoder()
df['artist'] = le.fit_transform(df['artist'].values)

In [9]:
def get_data(df, infer=False):
    if infer:
        return df['img_path'].apply(lambda p: os.path.join(data_path, p)).values
    return df['img_path'].apply(lambda p: os.path.join(data_path, p)).values, df['artist'].values

In [10]:
from torchvision.transforms import ToTensor


class CustomDataset(Dataset):
    def __init__(self, img_paths, labels, transforms=None):
        self.img_paths = img_paths
        self.labels = labels
        self.transforms = transforms if transforms else ToTensor()

    def __getitem__(self, index):
        img_path = self.img_paths[index]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transforms(image=image)['image']
        
        if self.labels is not None:
            label = self.labels[index]
            return image, label
        else:
            return image
    
    def __len__(self):
        return len(self.img_paths)

In [11]:
train_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE']*2,CFG['IMG_SIZE']*2),
    A.RandomCrop(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
    A.Transpose(p=0.5), # 행렬 스왑
    A.HorizontalFlip(p=0.5), # 좌우 반전
    A.VerticalFlip(p=0.5), # 상하 반전
    # A.ShiftScaleRotate(p=0.5), # 랜덤하게 옮기고, scale, 회전
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
    A.CoarseDropout(p=0.5),
    ToTensorV2()
])

validation_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE']*2,CFG['IMG_SIZE']*2),
    A.RandomCrop(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
    ToTensorV2()
])

test_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
    ToTensorV2()
])

In [12]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2 ** 32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
g = torch.Generator()
g.manual_seed(0)

In [13]:
class CAFormerModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(CAFormerModel, self).__init__()
        self.backbone = timm.create_model('caformer_s18.sail_in22k_ft_in1k_384', pretrained=True, num_classes=0)
        self.classifier = nn.Sequential(
            nn.LayerNorm(512),
            nn.GELU(),
            nn.Dropout(p=0.4),
            nn.Linear(512, num_classes)
        )
            
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [14]:
from torchsummary import summary

model = CAFormerModel()
summary(model, (3,CFG['IMG_SIZE'],CFG['IMG_SIZE']))

Layer (type:depth-idx)                        Output Shape              Param #
├─MetaFormer: 1-1                             [-1, 512]                 --
|    └─Stem: 2-1                              [-1, 64, 68, 68]          --
|    |    └─Conv2d: 3-1                       [-1, 64, 68, 68]          9,472
|    |    └─LayerNorm2dNoBias: 3-2            [-1, 64, 68, 68]          64
|    └─Sequential: 2-2                        [-1, 512, 9, 9]           --
|    |    └─MetaFormerStage: 3-3              [-1, 64, 68, 68]          166,668
|    |    └─MetaFormerStage: 3-4              [-1, 128, 34, 34]         702,156
|    |    └─MetaFormerStage: 3-5              [-1, 320, 17, 17]         11,439,826
|    |    └─MetaFormerStage: 3-6              [-1, 512, 9, 9]           10,918,726
|    └─Sequential: 2                          []                        --
|    |    └─SelectAdaptivePool2d: 3-7         [-1, 512, 1, 1]           --
|    |    └─LayerNorm2d: 3-8                  [-1, 512, 1, 1]     

Layer (type:depth-idx)                        Output Shape              Param #
├─MetaFormer: 1-1                             [-1, 512]                 --
|    └─Stem: 2-1                              [-1, 64, 68, 68]          --
|    |    └─Conv2d: 3-1                       [-1, 64, 68, 68]          9,472
|    |    └─LayerNorm2dNoBias: 3-2            [-1, 64, 68, 68]          64
|    └─Sequential: 2-2                        [-1, 512, 9, 9]           --
|    |    └─MetaFormerStage: 3-3              [-1, 64, 68, 68]          166,668
|    |    └─MetaFormerStage: 3-4              [-1, 128, 34, 34]         702,156
|    |    └─MetaFormerStage: 3-5              [-1, 320, 17, 17]         11,439,826
|    |    └─MetaFormerStage: 3-6              [-1, 512, 9, 9]           10,918,726
|    └─Sequential: 2                          []                        --
|    |    └─SelectAdaptivePool2d: 3-7         [-1, 512, 1, 1]           --
|    |    └─LayerNorm2d: 3-8                  [-1, 512, 1, 1]     

In [15]:
def clear_mem():
    gc.collect()
    torch.cuda.empty_cache()

In [16]:
class EarlyStopping:
    def __init__(self, patience=10, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, score):
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            print(f'Best F1 score from now: {self.best_score}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0
        
        return self.early_stop

In [17]:
def rand_bbox(size, lam):   # size : [B, C, W, H]
    W = size[2] # 이미지의 width
    H = size[3] # 이미지의 height
    cut_rat = np.sqrt(1. - lam)  # 패치 크기의 비율 정하기
    cut_w = np.int32(W * cut_rat)  # 패치의 너비
    cut_h = np.int32(H * cut_rat)  # 패치의 높이

    # uniform
    # 기존 이미지의 크기에서 랜덤하게 값을 가져옵니다.(중간 좌표 추출)
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    # 패치 부분에 대한 좌표값을 추출합니다.
    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [18]:
import math
from torch.optim.lr_scheduler import LRScheduler

class CosineAnnealingWarmUpRestarts(LRScheduler):
    def __init__(self, optimizer, T_0, T_mult=1, eta_max=0.1, T_up=0, gamma=1., last_epoch=-1, T_hold=5e-5):
        if T_0 <= 0 or not isinstance(T_0, int):
            raise ValueError("Expected positive integer T_0, but got {}".format(T_0))
        if T_mult < 1 or not isinstance(T_mult, int):
            raise ValueError("Expected integer T_mult >= 1, but got {}".format(T_mult))
        if T_up < 0 or not isinstance(T_up, int):
            raise ValueError("Expected positive integer T_up, but got {}".format(T_up))
        self.T_0 = T_0
        self.T_mult = T_mult
        self.base_eta_max = eta_max
        self.eta_max = eta_max
        self.T_up = T_up
        self.T_i = T_0
        self.gamma = gamma
        self.cycle = 0
        self.T_cur = last_epoch
        self.T_hold = T_hold # custom
        super(CosineAnnealingWarmUpRestarts, self).__init__(optimizer, last_epoch)
    
    def get_lr(self):
        if self.T_cur == -1:
            return self.base_lrs
        elif self.T_cur < self.T_up:
            return [(self.eta_max - base_lr)*self.T_cur / self.T_up + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.eta_max - base_lr) * (1 + math.cos(math.pi * (self.T_cur-self.T_up) / (self.T_i - self.T_up))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.T_cur = self.T_cur + 1
            if self.T_cur >= self.T_i:
                self.cycle += 1
                self.T_cur = self.T_cur - self.T_i
                self.T_i = (self.T_i - self.T_up) * self.T_mult + self.T_up
        else:
            if epoch >= self.T_0:
                if self.T_mult == 1:
                    self.T_cur = epoch % self.T_0
                    self.cycle = epoch // self.T_0
                else:
                    n = int(math.log((epoch / self.T_0 * (self.T_mult - 1) + 1), self.T_mult))
                    self.cycle = n
                    self.T_cur = epoch - self.T_0 * (self.T_mult ** n - 1) / (self.T_mult - 1)
                    self.T_i = self.T_0 * self.T_mult ** (n)
            else:
                self.T_i = self.T_0
                self.T_cur = epoch
                
        self.eta_max = self.base_eta_max * (self.gamma**self.cycle)
        if self.eta_max < self.T_hold:
            self.eta_max = self.T_hold
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

In [19]:
time_now = datetime.now()
run_id = time_now.strftime("%Y%m%d%H%M%S")
os.makedirs(os.path.join(data_path, f'./runs/{run_id}'), exist_ok=True)
print(f'{run_id=}')

run_id='20231227220443'


In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

In [21]:
def train(epoch, model, optimizer, criterion, train_loader, device, beta=1, cutmix_prob=0.5, lr_scheduler=None):
    model.train()
    train_loss = []
    lr_list = []
    bar = tqdm(enumerate(train_loader), total = len(train_loader), desc='Train Loop')
    for idx, (img, label) in bar:
        img, label = img.float().to(device), label.long().to(device)
        
        optimizer.zero_grad()
        r = np.random.rand(1)
        if beta > 0 and r < cutmix_prob:
            lam = np.random.beta(beta, beta)
            if lam < 0.5:
                lam = lam/2
            else:
                lam = 0.5 + lam/2
            rand_index = torch.randperm(img.size()[0]).cuda()
            target_a = label
            target_b = label[rand_index]
            bbx1, bby1, bbx2, bby2 = rand_bbox(img.size(), lam)
            img[:, :, bbx1:bbx2, bby1:bby2] = img[rand_index, :, bbx1:bbx2, bby1:bby2]

            lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (img.size()[-1] * img.size()[-2]))

            model_pred = model(img)
            loss = criterion(model_pred, target_a) * lam + criterion(model_pred, target_b) * (1. - lam)
        else:
            model_pred = model(img)
            loss = criterion(model_pred, label)
        
        loss.backward()
        if ((epoch-1)*len(train_loader) + idx)%10 == 0:
            lr_list.append(optimizer.param_groups[0]['lr'])
        optimizer.step()
        train_loss.append(loss.item())
        bar.set_postfix(train_loss = f'{loss.item():.4f}', lr = f"{optimizer.param_groups[0]['lr']:.6f}")
        if lr_scheduler:
            lr_scheduler.step()
    return np.mean(train_loss), lr_list

In [22]:
def competition_metric(true, pred):
    return f1_score(true, pred, average="macro")

def validation(model, criterion, test_loader, device):
    model.eval()
    
    model_preds = []
    true_labels = []
    
    val_loss = []
    
    with torch.no_grad():
        for img, label in iter(test_loader):
            img, label = img.float().to(device), label.long().to(device)
            
            model_pred = model(img)
            
            loss = criterion(model_pred, label)
            
            val_loss.append(loss.item())
            
            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += label.detach().cpu().numpy().tolist()
        
    val_f1 = competition_metric(true_labels, model_preds)
    return np.mean(val_loss), val_f1

In [23]:
def train_epoch(k, model, optimizer, train_loader, test_loader, lr_scheduler, device):
    model.to(device)

    criterion = FocalLoss(alpha=1, gamma=2).to(device)
    early_stopping = EarlyStopping(patience=CFG['PATIENCE'], verbose=True)

    best_score = 0
    lr_list = []
    train_loss_list, val_loss_list = [], []

    for epoch in range(1,CFG["EPOCHS"]+1):
        tr_loss, lr_ = train(epoch, model, optimizer, criterion, train_loader, device, beta=1, lr_scheduler=lr_scheduler)
        val_loss, val_score = validation(model, criterion, test_loader, device)
        train_loss_list.append(tr_loss)
        val_loss_list.append(val_loss)

        if lr_scheduler is not None:
            lr_list.extend(lr_)

        if best_score < val_score:
            print(f'**Epoch [{k}-{epoch}], lr : [{lr_[-1]:.6f}], Train Loss : [{tr_loss:.4f}], Val Loss : [{val_loss:.4f}], Val F1 Score : [{val_score:.4f}]')
            best_score = val_score
            torch.save(model, os.path.join(data_path, f'runs/{run_id}/best_model_{k}.pt'))
        else:
            print(f'Epoch [{k}-{epoch}], lr : [{lr_[-1]:.6f}], Train Loss : [{tr_loss:.4f}], Val Loss : [{val_loss:.4f}], Val F1 Score : [{val_score:.4f}]')
        clear_mem()
        if early_stopping(val_score):
            print(f'Epoch [{k}-{epoch}], early stopping')
            break
    if lr_list:
        return (train_loss_list, val_loss_list, best_score, lr_list)
    else:
        return (train_loss_list, val_loss_list, best_score, None)

In [24]:
def k_fold(k): # k-fold
    skf = StratifiedKFold(n_splits=k, shuffle=False)
    f1_sum = 0
    
    for k_, (train_index, valid_index) in enumerate(skf.split(df, df['artist'])):
        print(f'{k_}-fold start')
        
        class_counts = df.loc[train_index, 'artist'].value_counts(sort=False).to_dict()
        num_samples = sum(class_counts.values())
        labels = df.loc[train_index, 'artist'].to_list()
        class_weights = {l:round(num_samples/(class_counts[l]**0.5), 2) for l in class_counts.keys()}
        weights = [class_weights[labels[i]] for i in range(int(num_samples))]
        sampler = torch.utils.data.WeightedRandomSampler(torch.DoubleTensor(weights), int(num_samples))

        train_img_paths, train_labels = get_data(df.iloc[train_index])
        val_img_paths, val_labels = get_data(df.iloc[valid_index])
        train_dataset = CustomDataset(train_img_paths, train_labels, train_transform)
        train_loader = DataLoader(
            train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, #sampler=sampler,
            worker_init_fn=seed_worker, generator=g, num_workers=0
        )
        
        val_dataset = CustomDataset(val_img_paths, val_labels, validation_transform)
        val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, worker_init_fn=seed_worker, generator=g, num_workers=0)

        model = CAFormerModel()
        model.eval()
        optimizer = torch.optim.AdamW(params = model.parameters(), lr = CFG['LEARNING_RATE'])
        # lr : 5epochs 동안 0.001->CFG['LEARNING_RATE']
        lr_scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=CFG['WARMUP']*len(train_loader), T_mult=1, eta_max=0.0001, T_up=50, gamma=0.5, T_hold=CFG['LEARNING_RATE']*10)
        _, _, f1_score, _ = train_epoch(k_, model, optimizer, train_loader, val_loader, lr_scheduler, device)
        f1_sum += f1_score
    return f1_sum/k

In [25]:
f1_score = k_fold(CFG['K-FOLD'])
print(f1_score)

0-fold start


Train Loop:   0%|          | 0/296 [00:00<?, ?it/s]

**Epoch [0-1], lr : [0.000095], Train Loss : [2.7250], Val Loss : [1.6424], Val F1 Score : [0.3115]


Train Loop:   0%|          | 0/296 [00:00<?, ?it/s]

**Epoch [0-2], lr : [0.000078], Train Loss : [1.5610], Val Loss : [1.0739], Val F1 Score : [0.4753]


...

Epoch [4-32], lr : [0.000004], Train Loss : [0.2298], Val Loss : [0.4526], Val F1 Score : [0.7918]
EarlyStopping counter: 10 out of 12
Best F1 score from now: 0.8259727272520294


Train Loop:   0%|          | 0/296 [00:00<?, ?it/s]

Epoch [4-33], lr : [0.000003], Train Loss : [0.2107], Val Loss : [0.4728], Val F1 Score : [0.7952]
EarlyStopping counter: 11 out of 12
Best F1 score from now: 0.8259727272520294


Train Loop:   0%|          | 0/296 [00:00<?, ?it/s]

Epoch [4-34], lr : [0.000002], Train Loss : [0.2273], Val Loss : [0.4379], Val F1 Score : [0.8147]
EarlyStopping counter: 12 out of 12
Best F1 score from now: 0.8259727272520294
Epoch [4-34], early stopping
0.8307654319362665


In [37]:
test_df = pd.read_csv(os.path.join(data_path, './test.csv'))
test_df.head()

,id,img_path
0,TEST_00000,./test/TEST_00000.jpg
1,TEST_00001,./test/TEST_00001.jpg
2,TEST_00002,./test/TEST_00002.jpg
3,TEST_00003,./test/TEST_00003.jpg
4,TEST_00004,./test/TEST_00004.jpg


In [38]:
test_img_paths = get_data(test_df, infer=True)

In [39]:
test_dataset = CustomDataset(test_img_paths, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [40]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    
    model_preds = []
    
    with torch.no_grad():
        for idx, img in enumerate(test_loader):
            img = img.float().to(device)
            
            model_pred = model(img).detach().cpu()
            model_pred = F.softmax(model_pred, dim=1)
            model_preds.extend(model_pred.numpy().tolist())
    
    print('Done.')
    return model_preds

In [41]:
result_df = pd.DataFrame(np.zeros((test_df.shape[0], len(le.classes_))))
for k_ in range(CFG['K-FOLD']):
    checkpoint = os.path.join(data_path, f'runs/{run_id}/best_model_{k_}.pt')
    print(f'{k_}-fold CHECKPOINT LOADED: {checkpoint}')
    infer_model = torch.load(checkpoint)
    ret = inference(infer_model, test_loader, device)
    result_df += pd.DataFrame(ret)

0-fold CHECKPOINT LOADED: ../../data/runs/20231227220443/best_model_0.pt
Done.
1-fold CHECKPOINT LOADED: ../../data/runs/20231227220443/best_model_1.pt
Done.
2-fold CHECKPOINT LOADED: ../../data/runs/20231227220443/best_model_2.pt
Done.
3-fold CHECKPOINT LOADED: ../../data/runs/20231227220443/best_model_3.pt
Done.
4-fold CHECKPOINT LOADED: ../../data/runs/20231227220443/best_model_4.pt
Done.


In [48]:
result_df.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.003708,0.009087,0.004699,0.003643,0.002822,0.004589,0.005564,0.003409,0.001804,0.047481,...,0.007448,0.005319,0.011182,0.004573,0.005545,0.006051,0.020833,0.002266,0.036456,0.006299
1,0.004031,0.003125,4.741344,0.003319,0.002471,0.002338,0.001299,0.002439,0.016347,0.003524,...,0.002693,0.001955,0.003131,0.007745,0.001894,0.002673,0.003984,0.002367,0.011093,0.001583
2,0.011057,0.013050,0.004738,0.005612,0.006272,0.003512,4.093072,0.004018,0.005727,0.109418,...,0.006212,0.109290,0.018979,0.010112,0.031653,0.017325,0.088354,0.004858,0.016058,0.005813
3,4.820197,0.003335,0.002545,0.002804,0.002460,0.001580,0.001400,0.001566,0.001247,0.002398,...,0.015391,0.002335,0.003257,0.002933,0.002055,0.002677,0.003872,0.001384,0.016738,0.002697
4,0.004187,0.006125,0.042994,0.006308,0.006542,0.003256,0.000850,0.004717,0.002436,0.003186,...,0.003429,0.001683,0.003801,0.005928,0.002489,0.002833,0.005225,0.005082,1.949325,0.002441


In [42]:
preds = result_df.idxmax(axis=1)

In [43]:
preds = le.inverse_transform(preds)

In [44]:
submit = pd.read_csv(os.path.join(data_path, './sample_submission.csv'))

In [45]:
submit['artist'] = preds

In [46]:
submit.head()

,id,artist
0,TEST_00000,Edgar Degas
1,TEST_00001,Amedeo Modigliani
2,TEST_00002,Caravaggio
3,TEST_00003,Albrecht Du rer
4,TEST_00004,Pablo Picasso


In [47]:
submit.to_csv(os.path.join(data_path, f"./submit_{CFG['FILENAME']}.csv"), index=False)